In [ ]:
%matplotlib inline

import numpy as np
import os
import tarfile
import urllib
from IPython.display import display, Image
from scipy import ndimage
import cPickle as pickle
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')
import pandas as pd

In [ ]:
def load_data(file_name):
     with open(file_name, 'rb') as f:
        data=pickle.load(f)
        labels=data["label"].astype(np.int32)
        images=data["image"]
        del data
        return images,labels
loaded_train_data,loaded_train_labels=load_data("../train.pickle")
loaded_extra_data,loaded_extra_labels=load_data("../extra.pickle")
test_data,test_labels=load_data("../test.pickle")
print("train:",loaded_train_data.shape,loaded_train_labels.shape)
print("extra:",loaded_extra_data.shape,loaded_extra_labels.shape)
print("test:",test_data.shape,test_labels.shape)

In [ ]:
data=np.concatenate((loaded_train_data,loaded_extra_data,test_data))
labels=np.concatenate((loaded_train_labels,loaded_extra_labels,test_labels))
print("data:",data.shape,labels.shape)

In [ ]:
image_size=32
def display_image(image):
    plt.figure()
    plt.imshow(image)
    plt.show()
    
def display_images(data,labels,name=None,num_images=5):
    print ("showing some labels and images for %s"%name)  
    print ("data shape: ",data.shape)
    print("labels shape: ",labels.shape)
    total_images=data.shape[0]
    for i in np.random.choice(total_images,num_images):
        print([l for l in labels[i] if l!=10])   
        display_image(data[i].reshape(image_size,image_size))
            
display_images(data,labels,"data")

In [ ]:
pd_labels=pd.DataFrame(data=labels,columns=range(1,7))

In [ ]:
pd_labels[:5]

In [ ]:
valid_digits=pd_labels.apply(lambda x:x!=10)

In [ ]:
valid_digits[:5]

In [ ]:
digit_lengths=valid_digits.sum(axis=1)

In [ ]:
digit_lengths[:5]

In [ ]:
print("total digits: ",digit_lengths.sum())

In [ ]:
print("average number of digits per image",digit_lengths.mean())

In [ ]:
grouped=digit_lengths.groupby(digit_lengths.values)
print("digit length counts", grouped.count())

In [ ]:
digit_counts=pd.DataFrame()
for i in range(10):
    digit_counts[str(i)]=pd_labels.apply(lambda x:x==i).sum(axis=1)

In [ ]:
digit_counts[:5]

In [ ]:
number_of_mages_with_at_least_one=digit_counts.apply(lambda x:x!=0).sum(axis=0)

In [ ]:
number_of_mages_with_at_least_one

In [ ]:
number_of_mages_with_at_least_one.plot.bar(title="Number of images in which each digit appears").set(xlabel="Digit", ylabel="Number of images")

In [ ]:
number_of_digit_occurences=digit_counts.sum(axis=0)

In [ ]:
number_of_digit_occurences.plot.bar(title="Number occurences of each digit").set(xlabel="Digit", ylabel="Number of occurences")